In [5]:
!pip install tensorflow pandas matplotlib sklearn

  Using cached tensorflow-2.12.0-cp310-cp310-win_amd64.whl (1.9 kB)
  Using cached tensorflow_intel-2.12.0-cp310-cp310-win_amd64.whl (272.8 MB)



[notice] A new release of pip is available: 23.1 -> 23.1.2
[notice] To update, run: C:\Users\ZIYAD\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [7]:
import os
import pandas as pd
import tensorflow as tf
import numpy as np


In [8]:
df = pd.read_csv(os.path.join('train.csv','train.csv'))

In [9]:
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\r\nWhy the edits made under my use...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\r\nMore\r\nI can't make any real suggestions...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [10]:
from tensorflow.keras.layers import TextVectorization

In [11]:
x= df['comment_text']
y= df[df.columns[2:]].values

In [12]:
MAX_WORDS = 500000 # number of words in the vocab

In [13]:
vectorizer = TextVectorization(max_tokens=MAX_WORDS,output_sequence_length=1800,output_mode='int')

In [14]:
vectorizer.adapt(x.values)

In [15]:
vectorized_text= vectorizer(x.values)

In [16]:
#MCSHBAP - map, cache, shuffle, batch, prefetch
dataset = tf.data.Dataset.from_tensor_slices((vectorized_text,y))
dataset = dataset.cache()
dataset = dataset.shuffle(160000)
dataset = dataset.batch(16)
dataset = dataset.prefetch(8)


In [17]:
batch_x, batch_y = dataset.as_numpy_iterator().next()

In [18]:
train = dataset.take(int(len(dataset)*.7))
val = dataset.skip(int(len(dataset)*.7)).take(int(len(dataset)*.2))
test = dataset.skip(int(len(dataset)*.9)).take(int(len(dataset)*.1))


In [19]:
train_generator = train.as_numpy_iterator()

In [20]:
train_generator.next()

(array([[  46,   33,   15, ...,    0,    0,    0],
        [  27,  132,    4, ...,    0,    0,    0],
        [ 124,    7,   13, ...,    0,    0,    0],
        ...,
        [ 202,    2, 2063, ...,    0,    0,    0],
        [ 253,    2, 4495, ...,    0,    0,    0],
        [ 425, 1025,   40, ...,    0,    0,    0]], dtype=int64),
 array([[0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [1, 1, 1, 0, 1, 0]], dtype=int64))

In [21]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Bidirectional, Embedding , Dense

In [22]:
model = Sequential()
model.add(Embedding(MAX_WORDS+1,32))
model.add(Bidirectional(LSTM(32,activation='tanh')))
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(6, activation='sigmoid'))

In [23]:
model.compile(loss='BinaryCrossentropy',optimizer='Adam')

In [24]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          16000032  
                                                                 
 bidirectional (Bidirectiona  (None, 64)               16640     
 l)                                                              
                                                                 
 dense (Dense)               (None, 128)               8320      
                                                                 
 dense_1 (Dense)             (None, 256)               33024     
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dense_3 (Dense)             (None, 6)                 774       
                                                        

In [43]:
history= model.fit(train, epochs=1, validation_data=val)

1118/6981 [===>..........................] - ETA: 1:13:59 - loss: 0.0909

KeyboardInterrupt: 

In [26]:
history.history

NameError: name 'history' is not defined

In [27]:
from matplotlib import pyplot as plt

In [28]:
plt.figure(figsize=(8,5))
pd.DataFrame(history.history).plot()
plt.show()

NameError: name 'history' is not defined

<Figure size 800x500 with 0 Axes>

In [33]:
input_text = vectorizer('you are shit')

In [34]:
batch = test.as_numpy_iterator().next()

In [35]:
batch_x, batch_y = test.as_numpy_iterator().next()

In [36]:
(model.predict(batch_x)>0.5).astype(int)

1/1 [==============================] - 0s 105ms/step


array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])

In [37]:
res = model.predict(np.expand_dims(input_text,0))

1/1 [==============================] - 0s 133ms/step


In [38]:
from tensorflow.keras.metrics import Precision, Recall, CategoricalAccuracy

In [39]:
pre = Precision()
re = Recall()
acc= CategoricalAccuracy()

In [44]:
for batch in test.as_numpy_iterator():
    #unpack the batch
    X_true, y_true = batch
    #make a prediction    
    yhat = model.predict(X_true)
    #flatten the predictions
    y_true= y_true.flatten()
    yhat = yhat.flatten()
    
    pre.update_state(y_true,yhat)
    re.update_state(y_true,yhat)
    acc.update_state(y_true,yhat)


1/1 [==============================] - 0s 112ms/step


In [46]:
print(f'Precision :{pre.result().numpy()}, Recall:{re.result().numpy()}, Accuracy:{acc.result().numpy()}')

Precision :0.7918848395347595, Recall:0.2747918367385864, Accuracy:0.2300596833229065
